# Folk $n$-gram Analysis (FONN)

*FONN* (pronounced "fun") is an Irish (*Gaeilge*) word for "tune".

In this corpus we present two Polifonia components:

1. aeflkjaef
2. aeflkeajflakjf

This Demo notebook will demonstrate how we use them so far.

### Prerequisites

* In `<basepath>/MIDI` we should have a corpus of folk tunes in MIDI format. By default `basepath` is `./corpus/`. If the corpus is elsewhere, change `basepath` below. We will be writing outputs to subdirectories of `basepath`.

* Install the libraries *Feather*, *PyArrow*, *fastDamerauLevenshtein*, and *Music21*:

    `pip install feather music21 pyarrow fastDamerauLevenshtein`

In [1]:
basepath = "./corpus/"

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
from fastDamerauLevenshtein import damerauLevenshtein
import sys
sys.path.append("setup_corpus") # we should be able to remove this by making setup_corpus a proper module

These two Python scripts contain tools for reading the MIDI data, processing it to find the primary and secondary feature sequences, key-invariant sequences, and duration-weighted sequences.

In [3]:
from setup_corpus import setup_corpus
from corpus_processing_tools import Music21Corpus, MusicDataCorpus


Setting up constants.py lookup table for root assignment:
  note names  midi num  root num
0          C        60         0
1   C# or D-        61         1
2          D        62         2
3   D# or E-        63         3
4          E        64         4 




In [4]:
inpath = basepath + "/MIDI"
m21_corpus = Music21Corpus(inpath)

Input corpus contains 1224 melodies:

Tureengarbh Jig, The
Young And Stylish
Fun at the Fair
Buckley the Fiddler  (reel)
Tommy Coen's Reel
Lynch's Hornpipe
Tuttle's Reel
Patrick O'Connor's  (polka)
Patsy Tuohey's (reel)
Beggarman's Reel, The
Up and About In the Morning  (slide)
My Love Between Two Roses (reel)
Ladies Step Up to Tea!
Old Pigeon on the Gate, The
Curragh Races (reel), The
Mangan's Fancy (reel)
McDonagh's Reel 1
One of Tommy's  (hornpipe)
Listowel Fiddler  (slide), The
John Flynn's Jig
Callan Lasses (reel), The
Knocknaboul Reel, The
Jackson's Post-Chaise
Father Skehan's Jig
Tom Connors' Jig
Sailor's Bonnet (reel), The
Pomeroy Fiddler  (reel), The
Her Golden Hair Was Curling Down
Frog in the Puddle , The
Tap the Barrel (reel)
Trip to Durrow (reel), The
Humors of Derrycros(s)ane, The
Nine Mile House  (reel)
Duke of Leinster Hornpipe, The
Mason's Apron (reel), The
Pat O'Beirne's Favorite
Gooseberry Bush (reel), The
Carty's Reel
Brother Gildas' Jig
Hunter's Purse (reel), The
L

Fanning's Jig
When Sick Is It Tea You Want
Shamrock Jig, The
Tom Hill's (hornpipe)
Bresnahan's Frolics
Kiss the Maid Behind the Barrel (reel)
White Leaf (reel), The
New Miss McLeod (reel), The
Antrim Rose (reel)
Farnaught Jig , The
Silver Grove (hornpipe)  , The
Rakish Paddy (reel)
Flogging Reel
Morrison's (reel)
Bashful Bachelor 1 (hornpipe)
Micho's Tent ( ) (reel)
After the Hare  (reel)
Cloone (reel)
Mamma's Polka
Newport Lass, The
Martin Mulhaire's (reel)
Tumble the Tailor
Kilkenny Jig, The
Dan Sweeney's  1 (polka)
Basket of Turf, The(1)
Mother's Delight (reel)
Foghorn Hornpipe , The
High Road to Linton (reel)
Miss McGuinness'  (reel)
Brother's Pipes  (reel)
Piper's Lord McDonald (reel), The
Peacock's Feather 2 (hornpipe)
I Have a Bonnet (polka)
Greig's Pipes (reel)
Limerick Blacksmith  , The
Humors of Castlefinn (reel), The
Big Barney's Jig
Rough Road (reel) , The
Jaunting Car for Six
Mulloy's Reel
Butcher's March [iii], The
Mister Jones' Kerry Reel
Queen of the May (reel), The
Pad

Running the following cell will take about 15 minutes. It will produce many `csv` files under `<basepath>/feat_seq_data/note`, `<basepath>/feat_seq_data/accent`, `<basepath>/feat_seq_data/duration_weighted`. 

In [5]:
corpus = setup_corpus.SetupCorpus(m21_corpus)
corpus.generate_primary_feat_seqs()
corpus.setup_music_data_corpus()
corpus.run_simple_secondary_feature_sequence_calculations()
roots_path = basepath + "/roots.csv"
corpus.run_key_invariant_sequence_calulations(roots_path)
corpus.run_duration_weighted_sequence_calculations(['pitch', 'pitch_class'])
corpus.save_corpus(
    feat_seq_path=basepath + "/feat_seq_data/note",
    accents_path=basepath + "/feat_seq_data/accent",
    duration_weighted_path=basepath + "/feat_seq_data/duration_weighted"
)



/Users/jmmcd/opt/anaconda3/lib/python3.8/site-packages/music21/midi/translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=0, channel=10, data=0>; getting generic UnpitchedPercussion
  warnings.warn(
/Users/jmmcd/opt/anaconda3/lib/python3.8/site-packages/music21/midi/translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=1, channel=10, data=0>; getting generic UnpitchedPercussion
  warnings.warn(
/Users/jmmcd/opt/anaconda3/lib/python3.8/site-packages/music21/midi/translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=0, channel=10, data=0>; getting generic UnpitchedPercussion
  warnings.warn(
/Users/jmmcd/opt/anaconda3/lib/python3.8/site-packages/music21/midi/translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=0, channel=10, data=0>; g


1224 melodies successfully read to Music21 stream representation.
1224 melodies successfully converted to feature sequence representation.

Input corpus contains 1224 melodies:

Tureengarbh Jig, The
Young And Stylish
Fun at the Fair
Buckley the Fiddler  (reel)
Tommy Coen's Reel
Lynch's Hornpipe
Tuttle's Reel
Patrick O'Connor's  (polka)
Patsy Tuohey's (reel)
Beggarman's Reel, The
Up and About In the Morning  (slide)
My Love Between Two Roses (reel)
Ladies Step Up to Tea!
Old Pigeon on the Gate, The
Curragh Races (reel), The
Mangan's Fancy (reel)
McDonagh's Reel 1
One of Tommy's  (hornpipe)
Listowel Fiddler  (slide), The
John Flynn's Jig
Callan Lasses (reel), The
Knocknaboul Reel, The
Jackson's Post-Chaise
Father Skehan's Jig
Tom Connors' Jig
Sailor's Bonnet (reel), The
Pomeroy Fiddler  (reel), The
Her Golden Hair Was Curling Down
Frog in the Puddle , The
Tap the Barrel (reel)
Trip to Durrow (reel), The
Humors of Derrycros(s)ane, The
Nine Mile House  (reel)
Duke of Leinster Hornpipe, Th


MusicDataCorpus.rescale_corpus_durations() sample outputs for Tureengarbh Jig, The:

Sample feature sequence output (note-event level):
  MIDI_note  onset  duration velocity
0        71    0.0       1.0      105
1        69    0.5       1.0      105
2        71    1.0       1.0       80
3        73    1.5       1.0       80
4        74    2.0       1.0       95

Sample feature sequence output (accent-level):
   MIDI_note  onset  duration velocity
0         71    0.0       1.0      105
1         69    0.5       1.0      105
4         74    2.0       1.0       95
7         81    3.5       1.0      105
10        81    5.0       2.0       95



MusicDataCorpus.rescale_corpus_onsets(): sample outputs for Tureengarbh Jig, The:

Sample feature sequence output (note-event level):
  MIDI_note  onset  duration velocity
0        71    0.0       1.0      105
1        69    1.0       1.0      105
2        71    2.0       1.0       80
3        73    3.0       1.0       80
4        74    4.0       1


MusicDataCorpus.calc_duration_weighted_pitch_classes(): sample outputs for Tureengarbh Jig, The:
             dur_weighted_pitch  dur_weighted_pitch_class
eighth_note                                              
0                             9                         9
1                             7                         7
2                             9                         9
3                            11                        11
4                            12                         0


Writing output: ./corpus//feat_seq_data/note/Tureengarbh Jig, The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Tureengarbh Jig, The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Tureengarbh Jig, The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Young And Stylish_note.csv
Writing output: ./corpus//feat_seq_data/accent/Young And Stylish_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Young And Stylish_duration_weight

Writing output: ./corpus//feat_seq_data/note/Paddy Fahy's Jig(1)_note.csv
Writing output: ./corpus//feat_seq_data/accent/Paddy Fahy's Jig(1)_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Paddy Fahy's Jig(1)_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Whole Chicken in the Soup (slide), The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Whole Chicken in the Soup (slide), The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Whole Chicken in the Soup (slide), The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Kishkeam Ladies , The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Kishkeam Ladies , The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Kishkeam Ladies , The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Scully Casey's #2_note.csv
Writing output: ./corpus//feat_seq_data/accent/Scully Casey's #2_accent.csv
Writing output: ./corpus//feat_se

...

Writing output: ./corpus//feat_seq_data/accent/Music Club (reel), The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Music Club (reel), The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Hut in the Bog (reel), The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Hut in the Bog (reel), The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Hut in the Bog (reel), The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Lynch's (reel)_note.csv
Writing output: ./corpus//feat_seq_data/accent/Lynch's (reel)_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Lynch's (reel)_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Owenmore Polka , The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Owenmore Polka , The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Owenmore Polka , The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Crib o

Writing output: ./corpus//feat_seq_data/note/My Love is Such a Fair One (reel)_note.csv
Writing output: ./corpus//feat_seq_data/accent/My Love is Such a Fair One (reel)_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/My Love is Such a Fair One (reel)_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Dinny O'Brien's (reel)_note.csv
Writing output: ./corpus//feat_seq_data/accent/Dinny O'Brien's (reel)_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Dinny O'Brien's (reel)_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Stage Reel, The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Stage Reel, The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Stage Reel, The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Blue Pantaloons (reel), The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Blue Pantaloons (reel), The_accent.csv
Writing output: ./corpus//feat_seq_da

Writing output: ./corpus//feat_seq_data/duration_weighted/Oak Tree (reel), The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Dennis Cronin's Jig_note.csv
Writing output: ./corpus//feat_seq_data/accent/Dennis Cronin's Jig_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Dennis Cronin's Jig_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Laura's Delight_note.csv
Writing output: ./corpus//feat_seq_data/accent/Laura's Delight_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Laura's Delight_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Paddy Fahy's Reel 2_note.csv
Writing output: ./corpus//feat_seq_data/accent/Paddy Fahy's Reel 2_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Paddy Fahy's Reel 2_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Autumn Leaves (reel), The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Autumn Leaves (reel), The_ac

Writing output: ./corpus//feat_seq_data/note/Dillon's Reel_note.csv
Writing output: ./corpus//feat_seq_data/accent/Dillon's Reel_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Dillon's Reel_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Tinker's Apron (reel), The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Tinker's Apron (reel), The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Tinker's Apron (reel), The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Daniel of the Sun_note.csv
Writing output: ./corpus//feat_seq_data/accent/Daniel of the Sun_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Daniel of the Sun_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Two and Sixpenny Girl_note.csv
Writing output: ./corpus//feat_seq_data/accent/Two and Sixpenny Girl_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Two and Sixpenny Girl_duration_we

Writing output: ./corpus//feat_seq_data/accent/Kilcummin Slide_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Kilcummin Slide_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Johnny's Gone to France (reel)_note.csv
Writing output: ./corpus//feat_seq_data/accent/Johnny's Gone to France (reel)_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Johnny's Gone to France (reel)_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Game Cock (reel), The_note.csv
Writing output: ./corpus//feat_seq_data/accent/Game Cock (reel), The_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Game Cock (reel), The_duration_weighted.csv
Writing output: ./corpus//feat_seq_data/note/Willy Reidy's #2  (polka)_note.csv
Writing output: ./corpus//feat_seq_data/accent/Willy Reidy's #2  (polka)_accent.csv
Writing output: ./corpus//feat_seq_data/duration_weighted/Willy Reidy's #2  (polka)_duration_weighted.csv


For example, in `A Trip To Galway_note.csv`, the first few lines will be:

In [6]:
df = pd.read_csv("corpus/feat_seq_data/note/A Trip To Galway_note.csv")
df.head()

,Unnamed: 0,MIDI_note,onset,duration,velocity,interval,parsons_code,Parsons_cumsum,chromatic_root,pitch,pitch_class
0,0,74,0.0,1.0,105,0,0,0,4,10,10
1,1,71,1.0,1.0,105,-3,-1,-1,4,7,7
2,2,67,2.0,1.0,80,-4,-1,-2,4,3,3
3,3,64,3.0,1.0,80,-3,-1,-3,4,0,0
4,4,64,4.0,1.0,95,0,0,-3,4,0,0


Here, we see that [FIXME add here]

Next, we will calculate the most important $n$-grams in each tune, calculating importance using TF-IDF.

In [7]:
from ngram_tfidf_tools import NgramCorpus
from setup_ngrams_tfidf import SetupNgramsTfidf

Again, the following cell will take about 25 minutes to run.

In [8]:
inpath = basepath + "/feat_seq_data/accent"
feature = "pitch_class"
n_vals = list(range(5, 10))
feat_seq_corpus = NgramCorpus(inpath)
ngram_corpus = SetupNgramsTfidf(feat_seq_corpus, feature, n_vals)
ngram_corpus.extract_ngrams()
ngram_corpus.calculate_tfidf()
ngram_corpus.save_results(outpath=basepath + "/ngrams",
                          corpus_name='cre_pitch_class_accents')


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Primrose Girl (reel), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          62    0.0      1.00       105         0             0   
4          72    4.0      1.00        95        10             1   
8          71    8.0      1.00       105        -1            -1   
12         69   12.0      0.66        95        -2            -1   
17         62   16.0      1.00       105        -7            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               7     -5            7  
4                1               7      5            5  
8                0               7      4            4  
12              -1               7      2            2  
17              -2               7     -5            7  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Glenside Cottage (reel), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          69    0.0      1.00       105         0             0   
1          71    1.0      2.00       105         2             1   
4          62    5.0      1.00        95        -9            -1   
8          71    9.0      0.66       105         9             1   
13         71   13.0      1.00        95         0             0   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               4      5            5  
1                1               4      7            7  
4                0               4     -2           10  
8                1               4      7            7  
13               1               4      7            7  

Reading data from:
/Users/jmmcd/OneDrive - National University of Irel

/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Great Eastern Polka, The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          71    0.0       1.0       105         0             0   
2          74    2.0       1.0       105         3             1   
8          69   10.0       1.0       105        -5            -1   
14         64   18.0       1.0       105        -5            -1   
22         67   26.0       1.0       105         3             1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               7      4            4  
2                1               7      7            7  
8                0               7      2            2  
14              -1               7     -3            9  
22               0               7      0            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Brian o Laimhin (reel)_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          67    0.0       2.0       105         0             0   
3          79    4.0       1.0        95        12             1   
7          67    8.0       1.0       105       -12            -1   
11         72   12.0       1.0        95         5             1   
15         67   16.0       2.0       105        -5            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               7      0            0  
3                1               7     12            0  
7                0               7      0            0  
11               1               7      5            5  
15               0               7      0            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, G

12              -1               7      9            9  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Kildare Fancy (hornpipe), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          74    0.0      1.32       105         0             0   
2          69    2.0      1.32       105        -5            -1   
6          69    6.0      1.32        95         0             0   
10         69   10.0      1.32       105         0             0   
14         79   14.0      0.66        95        10             1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               2     12            0  
2               -1               2      7            7  
6               -1               2      7            7  
10              -1               2      7            7  
14               0               2     17            5  

Reading data

...

/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Mary Darby's_accent.csv
   MIDI_note  onset  duration  velocity  interval  parsons_code  \
0         62    0.0       2.0       105         0             0   
2         69    3.0       2.0        95         7             1   
4         71    6.0       1.0        95         2             1   
7         69    9.0       2.0        95        -2            -1   
9         62   12.0       2.0       105        -7            -1   

   Parsons_cumsum  chromatic_root  pitch  pitch_class  
0               0               2      0            0  
2               1               2      7            7  
4               2               2      9            9  
7               1               2      7            7  
9               0               2      0            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/fe


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Grand Gates of Annesbrook (reel), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          79    0.0       1.0       105         0             0   
2          76    2.0       1.0       105        -3            -1   
5          67    6.0       1.0        95        -9            -1   
9          76   10.0       1.0       105         9             1   
12         79   14.0       1.0        95         3             1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               9     10           10  
2               -1               9      7            7  
5               -2               9     -2           10  
9               -1               9      7            7  
12               0               9     10           10  

Reading data from:
/Users/jmmcd/OneDrive - National Universit


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/I Ne'er Shall Wean Her_accent.csv
   MIDI_note  onset  duration  velocity  interval  parsons_code  \
0         67    0.0       1.0       105         0             0   
1         64    1.0       1.0       105        -3            -1   
4         67    4.0       2.0        95         3             1   
6         64    7.0       1.0       105        -3            -1   
9         72   10.0       2.0        95         8             1   

   Parsons_cumsum  chromatic_root  pitch  pitch_class  
0               0               0      7            7  
1              -1               0      4            4  
4               0               0      7            7  
6              -1               0      4            4  
9               0               0     12            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub

9               1               9     12            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Sean McGovern's  (polka)_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          74    0.0       0.5       105         0             0   
2          69    1.0       1.0       105        -5            -1   
6          67    5.0       1.0       105        -2            -1   
11         69    9.0       1.0       105         2             1   
15         74   13.0       2.0       105         5             1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               2     12            0  
2               -1               2      7            7  
6               -2               2      5            5  
11              -1               2      7            7  
15               0               2     12            0  

Reading data from:

    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          69    0.0      0.66       105         0             0   
3          74    2.0      2.00       105         5             1   
6          62    6.0      1.32        95       -12            -1   
10         62   10.0      1.32       105         0             0   
14         74   14.0      2.00        95        12             1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               2      7            7  
3                1               2     12            0  
6                0               2      0            0  
10               0               2      0            0  
14               1               2     12            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Ranger (h'pipe), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          

Now we have TF-IDF scores for each $n$-gram for each tune. For example:

In [10]:
df = pd.read_csv("./corpus/ngrams/cre_pitch_class_accents_ngrams_freq.csv")
df.head()


,Unnamed: 0,ngram,"Primrose Girl (reel), The_accent_pitch_class_freq","Soldier's Joy, The_accent_pitch_class_freq","Sligo Jig, The_accent_pitch_class_freq","Rambling Connachtman (reel), The_accent_pitch_class_freq",Orange and Green_accent_pitch_class_freq,"London Lasses (reel), The_accent_pitch_class_freq",Dusty Miller_accent_pitch_class_freq,"Rainy Day Jig, The_accent_pitch_class_freq",...,Tear the Calico (reel)_accent_pitch_class_freq,Jack's Alive (reel)_accent_pitch_class_freq,"Tirnaskea Lasses (reel), The_accent_pitch_class_freq",Roaring Mary (reel)_accent_pitch_class_freq,"Master's Sporting Paddy (reel), The_accent_pitch_class_freq","Chattering Magpie (reel), The_accent_pitch_class_freq",Scully Casey's ( ) (hornpipe)_accent_pitch_class_freq,"Ranger (h'pipe), The_accent_pitch_class_freq",freq,idf
0,0,"(0, 0, 0, 0, 0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,595,6.225
1,1,"(0, 0, 0, 0, 0, 0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,494,6.410
2,2,"(0, 0, 0, 0, 0, 0, 0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,456,6.490
3,3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,435,6.537
4,4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,425,6.561


Explain a little of the above, or else print out something else to show what we have got from the ngrams.

In [14]:
df[["ngram", "A Trip To Galway_accent_pitch_class_freq", "freq", "idf"]]

,ngram,A Trip To Galway_accent_pitch_class_freq,freq,idf
0,"(0, 0, 0, 0, 0)",0,595,6.225
1,"(0, 0, 0, 0, 0, 0)",0,494,6.410
2,"(0, 0, 0, 0, 0, 0, 0)",0,456,6.490
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0,435,6.537
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0,425,6.561
...,...,...,...,...
495,"(7, 7, 0, 0, 4)",0,19,9.667
496,"(10, 10, 0, 2, 10)",0,19,9.667
497,"(7, 10, 7, 10, 7, 10)",0,19,9.667
498,"(0, 5, 0, 0, 4)",0,19,9.667


Finally, we will demonstrate some work-in-progress for calculating similarity between tunes, based on similarity between between their $n$-grams. This uses the Damerau-Levenshtein algorithm.

In [ ]:
from ngram_pattern_search import NgramSimilarity

In [ ]:
inpath = basepath + "/ngrams/cre_pitch_class_accents_ngrams_tfidf.ftr" # please check
pattern_search = NgramSimilarity(inpath)
pattern_search.extract_candidate_ngrams("Lord McDonald's (reel)", n=6, mode='idx', indices=[0, 1])
pattern_search.setup_test_corpus()
pattern_search.find_similar_patterns(edit_dist_threshold=1)
pattern_search.find_similar_tunes()
print(pattern_search.results)